## Convolutional Neural Network

0. Given 3 images and 12 = 3*4 kernals, each image has 4 kernals. Produces 4 outputs for these 3 images.
1. Reading the pixel and store it into a 6x6 matrix.
    - If replication padding, replicate the boundaries.
    - If zero-padding       , zero-pad the boundaries.
2. For every kernals of each image. 
    - For every possible windows in these 3 images.
        1. Generate their correspondent 3 kernal processed pixels.
        2. Sum these 3 pixels up.
        3. For the given operations, perform Relu, leaky Relu, sigmoid or tanh.
            - If 0, Relu
            - If 1, leaky Relu
            - If 2, sigmoid
            - If 3, tanh
        4. For the given processed kernal, store the output in the correspondent location of the    final kernal.
 

## Given 3 images and 12 = 3*4 kernals, each image has 4 kernals. Produces 4 outputs for 

In [61]:
import numpy as np
import random as rd
# Setting up global variables
NUM_OF_IMG      = 3
NUM_OF_KERNALS  = 4
IMG_SIZE        = 4
KERNAL_SIZE     = 3
PADDED_IMG_SIZE = 6
OUTPUT_IMG_SIZE = 8

#Randomly generating imgs and kernals
rd.seed(123)

imgs    = [np.array([[rd.randint(0,5) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], \
    dtype=object) for _ in range(NUM_OF_IMG)]

kernals = []
for _ in range(NUM_OF_IMG):
    kernals.append([np.array([[rd.randint(0,5) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], \
        dtype=object) for _ in range(NUM_OF_KERNALS)])


In [62]:
imgs

[array([[0, 2, 0, 3],
        [2, 0, 0, 3],
        [4, 4, 2, 2],
        [0, 1, 1, 2]], dtype=object),
 array([[4, 2, 5, 1],
        [1, 0, 3, 0],
        [4, 3, 0, 0],
        [2, 5, 3, 0]], dtype=object),
 array([[0, 0, 5, 1],
        [1, 0, 2, 3],
        [4, 3, 2, 3],
        [0, 2, 2, 4]], dtype=object)]

In [63]:
kernals[0]

[array([[3, 1, 4],
        [5, 4, 4],
        [2, 0, 3]], dtype=object),
 array([[5, 4, 3],
        [5, 4, 5],
        [5, 4, 3]], dtype=object),
 array([[4, 5, 3],
        [2, 4, 0],
        [5, 1, 5]], dtype=object),
 array([[0, 3, 5],
        [2, 1, 2],
        [2, 4, 3]], dtype=object)]

## Reading the image, according to OPs store it into 6x6 matrix

In [64]:
input_img = imgs[0]
padded_img = np.zeros((PADDED_IMG_SIZE,PADDED_IMG_SIZE))

ops = ['relu_replicate','leaky_relu_replicate','sigmoid_zero_pad','tanh_zero_pad']
operation = ops[0]

if operation == 'relu_replicate' or operation == 'leaky_relu_replicate':
    # Boundaries
    # Rows
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            pixel = input_img[i][j]
            # 4 corners
            if i == 0 and j == 0:
                padded_img[0][0] = pixel
                padded_img[0][1] = pixel
                padded_img[1][0] = pixel
                padded_img[1][1] = pixel
            elif i == 0 and j == IMG_SIZE-1:
                padded_img[1][4] = pixel
                padded_img[1][5] = pixel
                padded_img[0][4] = pixel
                padded_img[0][5] = pixel
            elif i == IMG_SIZE -1 and j == 0:
                padded_img[4][1] = pixel
                padded_img[4][0] = pixel
                padded_img[5][0] = pixel
                padded_img[5][1] = pixel
            elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
                padded_img[4][4] = pixel
                padded_img[4][5] = pixel
                padded_img[5][4] = pixel
                padded_img[5][5] = pixel
            elif i == 0:
                padded_img[0][j+1] = pixel
                padded_img[1][j+1] = pixel
            elif j == 0:
                padded_img[i+1][0] = pixel
                padded_img[i+1][1] = pixel
            elif i == IMG_SIZE -1:
                padded_img[4][j+1] = pixel
                padded_img[5][j+1] = pixel
            elif j == IMG_SIZE -1:
                padded_img[i+1][4] = pixel
                padded_img[i+1][5] = pixel
            else:
                padded_img[i+1][j+1] = pixel 
    
elif operation == 'sigmoid_zero_pad' or operation == 'tanh_zero_pad':
     for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            padded_img[i+1][j+1] = input_img[i][j]


padded_img
    

array([[0., 0., 2., 0., 3., 3.],
       [0., 0., 2., 0., 3., 3.],
       [2., 2., 0., 0., 3., 3.],
       [4., 4., 4., 2., 2., 2.],
       [0., 0., 1., 1., 2., 2.],
       [0., 0., 1., 1., 2., 2.]])

In [65]:
input_img

array([[0, 2, 0, 3],
       [2, 0, 0, 3],
       [4, 4, 2, 2],
       [0, 1, 1, 2]], dtype=object)

In [66]:
padded_img

array([[0., 0., 2., 0., 3., 3.],
       [0., 0., 2., 0., 3., 3.],
       [2., 2., 0., 0., 3., 3.],
       [4., 4., 4., 2., 2., 2.],
       [0., 0., 1., 1., 2., 2.],
       [0., 0., 1., 1., 2., 2.]])

In [67]:
def padding(input_img,operation):
    padded_img = np.zeros((PADDED_IMG_SIZE,PADDED_IMG_SIZE))

    if operation == 'relu_replicate' or operation == 'leaky_relu_replicate':
    # Boundaries
    # Rows
        for i in range(IMG_SIZE):
            for j in range(IMG_SIZE):
                pixel = input_img[i][j]
                # 4 corners
                if i == 0 and j == 0:
                    padded_img[0][0] = pixel
                    padded_img[0][1] = pixel
                    padded_img[1][0] = pixel
                    padded_img[1][1] = pixel
                elif i == 0 and j == IMG_SIZE-1:
                    padded_img[1][4] = pixel
                    padded_img[1][5] = pixel
                    padded_img[0][4] = pixel
                    padded_img[0][5] = pixel
                elif i == IMG_SIZE -1 and j == 0:
                    padded_img[4][1] = pixel
                    padded_img[4][0] = pixel
                    padded_img[5][0] = pixel
                    padded_img[5][1] = pixel
                elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
                    padded_img[4][4] = pixel
                    padded_img[4][5] = pixel
                    padded_img[5][4] = pixel
                    padded_img[5][5] = pixel
                elif i == 0:
                    padded_img[0][j+1] = pixel
                    padded_img[1][j+1] = pixel
                elif j == 0:
                    padded_img[i+1][0] = pixel
                    padded_img[i+1][1] = pixel
                elif i == IMG_SIZE -1:
                    padded_img[4][j+1] = pixel
                    padded_img[5][j+1] = pixel
                elif j == IMG_SIZE -1:
                    padded_img[i+1][4] = pixel
                    padded_img[i+1][5] = pixel
                else:
                    padded_img[i+1][j+1] = pixel 
    
    elif operation == 'sigmoid_zero_pad' or operation == 'tanh_zero_pad':
         for i in range(IMG_SIZE):
            for j in range(IMG_SIZE):
                padded_img[i+1][j+1] = input_img[i][j]
    
    return padded_img


## Process and store every padded img

In [68]:
padded_img = padding(input_img,operation)

In [69]:
input_img

array([[0, 2, 0, 3],
       [2, 0, 0, 3],
       [4, 4, 2, 2],
       [0, 1, 1, 2]], dtype=object)

In [70]:
padded_img

array([[0., 0., 2., 0., 3., 3.],
       [0., 0., 2., 0., 3., 3.],
       [2., 2., 0., 0., 3., 3.],
       [4., 4., 4., 2., 2., 2.],
       [0., 0., 1., 1., 2., 2.],
       [0., 0., 1., 1., 2., 2.]])

In [71]:
imgs[0]

array([[0, 2, 0, 3],
       [2, 0, 0, 3],
       [4, 4, 2, 2],
       [0, 1, 1, 2]], dtype=object)

In [72]:
for idx,img in enumerate(imgs):
    imgs[idx] = padding(img,operation)

In [73]:
imgs[0],imgs[1]

(array([[0., 0., 2., 0., 3., 3.],
        [0., 0., 2., 0., 3., 3.],
        [2., 2., 0., 0., 3., 3.],
        [4., 4., 4., 2., 2., 2.],
        [0., 0., 1., 1., 2., 2.],
        [0., 0., 1., 1., 2., 2.]]),
 array([[4., 4., 2., 5., 1., 1.],
        [4., 4., 2., 5., 1., 1.],
        [1., 1., 0., 3., 0., 0.],
        [4., 4., 3., 0., 0., 0.],
        [2., 2., 5., 3., 0., 0.],
        [2., 2., 5., 3., 0., 0.]]))

2. For every kernals of each image. 
    - For every possible windows in these 3 images.
        1. Generate their correspondent 3 kernal processed pixels.
        2. Sum these 3 pixels up.
        3. For the given operations, perform Relu, leaky Relu, sigmoid or tanh.
            - If 0, Relu
            - If 1, leaky Relu
            - If 2, sigmoid
            - If 3, tanh
        4. For the given processed kernal, store the output in the correspondent location of the    final kernal.

In [ ]:
for kernal in kernals:
    img0 = imgs[0]
    img1 = imgs[1]
    img2 = imgs[2]        